In [1]:
import numpy as np
import pandas as pd

from obj.Compos_DF import ComposDF
from obj.Compo_HTML import *
from obj.List import *
from obj.Block import *
from obj.Group import *
from obj.React import *
from obj.Page import *
from obj.Tree import *
import lib.draw as draw
from lib.list_item_gethering import gather_lists_by_pair_and_group

In [6]:
name = 'data/input/9'
compos = ComposDF(json_file=name + '.json', img_file=name + '.png')
img = compos.img.copy()
img_shape = compos.img_shape
img_re = cv2.resize(img, img_shape)

In [7]:
# ***Step 1*** repetitive list recognition
compos.repetitive_group_recognition()    # group_nontext, group_text
# check_valid_group_by_interleaving(compos.compos_dataframe)
# compos.pair_groups()                     # group_pair, pair_to, group
# compos.list_item_partition()             # list_item
# compos.visualize_block('group')
# compos.visualize_block('group_pair')

In [8]:
c = compos.compos_dataframe
grps = c.groupby('cluster_row_min')
max_id = max(grps.groups.keys())
for k in grps.groups:
    g = grps.groups[k]
    if len(g) == 1:
        continue
    gap_pre = c.loc[g[1]]['column_min'] - c.loc[g[0]]['column_min']
    section = [g[0]]
    
    for i in range(1, len(g)):
        gap = c.loc[g[i]]['column_min'] - c.loc[g[i - 1]]['column_min']
        # compos shouldn't be in same group as irregular gaps
        if gap_pre > gap * 2 or gap > gap_pre * 2:
            max_id += 1
            c.loc[section, 'cluster_row_min'] = max_id
            section = []
        else:
            section.append(g[i])

KeyError: 'cluster_row_min'

In [9]:
compos.visualize_block('cluster_row_min')

KeyError: 'cluster_row_min'

In [ ]:
for k in grps.groups:
    g = grps.groups[k]
    gaps = []
    for i in range(len(g) - 1):
        c_id = g[i]
        gaps.append(c.loc[g[i + 1]]['column_min'] - c.loc[g[i]]['column_min'])

    l_dic = {}
    for i,g in enumerate(gaps):
        grouped = False
        for key in l_dic:
            # valid grouped, update mean in the group
            if g < key < g*2 or key < g < key*2:
                l_dic[key].append(i)
                l_dic[np.mean((g, key))] = l_dic[key]
                del l_dic[key]
                grouped = True
                break
        if not grouped:
            l_dic[g] = [i]
    print(l_dic)

In [ ]:
compos.visualize('id')

In [ ]:
# ***Step 2*** mark compos in same group as a single list, mark compos in same group_pair as a multiple list
lists, non_listed_compos = gather_lists_by_pair_and_group(compos.compos_dataframe[1:])
generate_lists_html_css(lists)
visualize_lists(img_re, lists)

In [ ]:
# ***Step 3*** slice compos as blocks
compos_html = [li.list_obj for li in lists] + non_listed_compos
blocks, non_blocked_compos = slice_blocks(compos_html, 'v')
# visualize_CompoHTMLs(compos_html, img_re)
# visualize_blocks(blocks, img, img_shape)
# blocks[2].visualize_sub_blocks_and_compos(img_re, recursive=True)

In [ ]:
# ***Step 4*** assembly html and css as web page, and react program
html, css = export_html_and_css(blocks)
blk, index = export_react_program(blocks)
tree = export_tree(blocks)